In [124]:
import tarfile
import scanpy as sc
import scipy.io
import pandas as pd
import numpy as np

path = r"C:\Users\W\Desktop\Assignments\Data Science\project\repreduce 1\data_visual\adata_1"
adata = sc.read_h5ad(path)
adata.obs


,Unnamed: 0,0,perturbation
0,0,AAACATACACGTAC_dc3hLPS_A8,m_Rel_3
1,1,AAACATACATGTCG_dc3hLPS_A8,m_Nfkb1_2
2,2,AAACATACCAACTG_dc3hLPS_A8,m_Egr2_4
3,3,AAACATACTCCTTA_dc3hLPS_A8,control
4,4,AAACATACTCTCCG_dc3hLPS_A8,m_Runx1_2
...,...,...,...
32772,32772,TTTCTACTGACGAG_dc3hLPS_D9,m_Stat2_4
32773,32773,TTTCTACTTCTTAC_dc3hLPS_D9,control
32774,32774,TTTGACTGAAAAGC_dc3hLPS_D9,m_Rela_1
32775,32775,TTTGCATGCGACAT_dc3hLPS_D9,control


In [125]:
perturbation_names = adata.obs["perturbation"].unique()
print(perturbation_names)
print(adata.var)
import pandas as pd

perturbation_names = [gene for gene in perturbation_names if gene != "control"]

# Extract gene symbols from adata.var (remove ENSEMBL IDs)
adata.var["gene_symbol"] = adata.var["0"].str.split("_").str[-1]  # Extract last part after "_"

# Display example
print(adata.var["gene_symbol"].head())

# Example perturbed genes (from experimental data)


# Clean perturbed gene names by removing prefixes (`m_`) and suffixes (`_3`, `_2`, etc.)
cleaned_perturbed_genes = [gene.split("_")[1] for gene in perturbation_names]

print("✅ Cleaned perturbed genes:", cleaned_perturbed_genes)

# Check if each perturbed gene exists in the extracted gene symbols
matched_genes = [gene for gene in cleaned_perturbed_genes if gene in adata.var["gene_symbol"].values]

print("✅ Matched genes in adata.var:", matched_genes)


['m_Rel_3', 'm_Nfkb1_2', 'm_Egr2_4', 'control', 'm_Runx1_2', ..., 'm_Egr1_4', 'm_Hif1a_1', 'm_Irf1_1', 'm_Cebpb_1', 'm_Irf2_2']
Length: 58
Categories (58, object): ['control', 'm_Ahr_1', 'm_Ahr_3', 'm_Atf3_1', ..., 'm_Stat2_2', 'm_Stat2_3', 'm_Stat2_4', 'm_Stat3_3']
       Unnamed: 0                            0 gene_symbol   symbols
4976         4976  ENSMUSG00000028988_Ctnnbip1    Ctnnbip1  Ctnnbip1
7880         7880     ENSMUSG00000030505_Prmt3       Prmt3     Prmt3
9003         9003    ENSMUSG00000020260_Pofut2      Pofut2    Pofut2
8428         8428    ENSMUSG00000042289_Hsd3b7      Hsd3b7    Hsd3b7
1128         1128   ENSMUSG00000104206_Gm32250     Gm32250   Gm32250
...           ...                          ...         ...       ...
14321       14321     ENSMUSG00000021109_Hif1a       Hif1a     Hif1a
14986       14986      ENSMUSG00000042622_Maff        Maff      Maff
15812       15812     ENSMUSG00000022952_Runx1       Runx1     Runx1
16882       16882      ENSMUSG00000038418_E

In [126]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np
from fuzzywuzzy import process


def sketch_perturbation_genes(adata, perturbation_names):
    
    # Sketch the symbols in original AnnData gene names
    adata.var["symbols"] = adata.var["0"].str.split("_").str[-1]

    # Find the cleaned perturbed names "m_Rel_1": "Rel"
    cleaned_perturbation_genes = [gene.split("_")[1] for gene in perturbation_names if gene != "control"]

    # Match the cleaned perturbed names with symbols in original AnnData
    matched_genes = {}
    for gene in cleaned_perturbation_genes:
        match, score = process.extractOne(gene, adata.var["symbols"].values)
        
        # Find the best matches
        if score > 85 and gene.lower() in match.lower():
            matched_genes[gene] = match

    print("Matched Genes:", matched_genes)

    # Sketch the indices for the removed genes in the original AnnData
    remove_indices = adata.var.index[adata.var["symbols"].isin(matched_genes.values())].tolist()
    remove_indices = list(set(remove_indices))
    if not adata.var.index.is_unique:
        print("Warning: `adata.var.index` contains duplicate values!")
        adata.var = adata.var[~adata.var.index.duplicated(keep="first")]  # Keep only first occurrence
    # Extract the removed X and var in the AnnData for the missing genes
    adding_X = adata[:, remove_indices].X
    adding_var = adata.var.loc[remove_indices]

    # Update the AnnData with missing genes
    perturbation_adata = sc.AnnData(X=adding_X, obs=adata.obs.copy(), var=adding_var.copy())

    print(f"Recovered {len(remove_indices)} perturbation genes.")
    
    return perturbation_adata 

preturbed_adata = sketch_perturbation_genes(adata, perturbation_names)
print(adata.var)
preturbed_adata.var
perturbation_symbols = preturbed_adata.var["symbols"]
perturbation_index = preturbed_adata.var.index

gene_symbols = adata.var["symbols"]
gene_index = adata.var.index

Matched Genes: {'Rel': 'Rel', 'Nfkb1': 'Nfkb1', 'Egr2': 'Egr2', 'Runx1': 'Runx1', 'Maff': 'Maff', 'Stat2': 'Stat2', 'Hif1a': 'Hif1a', 'Stat1': 'Stat1', 'Rela': 'Rela', 'E2f4': 'E2f4', 'Irf4': 'Irf4', 'Irf2': 'Irf2', 'Relb': 'Relb', 'Stat3': 'Stat3', 'Spi1': 'Spi1', 'Ctcf': 'Ctcf', 'E2f1': 'E2f1', 'Cebpb': 'Cebpb', 'Irf1': 'Irf1', 'Ahr': 'Ahr', 'Junb': 'Junb', 'Atf3': 'Atf3', 'Ets2': 'Ets2', 'Egr1': 'Egr1'}
Recovered 24 perturbation genes.
       Unnamed: 0                            0 gene_symbol   symbols
4976         4976  ENSMUSG00000028988_Ctnnbip1    Ctnnbip1  Ctnnbip1
7880         7880     ENSMUSG00000030505_Prmt3       Prmt3     Prmt3
9003         9003    ENSMUSG00000020260_Pofut2      Pofut2    Pofut2
8428         8428    ENSMUSG00000042289_Hsd3b7      Hsd3b7    Hsd3b7
1128         1128   ENSMUSG00000104206_Gm32250     Gm32250   Gm32250
...           ...                          ...         ...       ...
14321       14321     ENSMUSG00000021109_Hif1a       Hif1a     Hif1a
14986

In [175]:
adata.obs

,Unnamed: 0,0,perturbation,symbols
0,0,AAACATACACGTAC_dc3hLPS_A8,m_Rel_3,Rel
1,1,AAACATACATGTCG_dc3hLPS_A8,m_Nfkb1_2,Nfkb1
2,2,AAACATACCAACTG_dc3hLPS_A8,m_Egr2_4,Egr2
3,3,AAACATACTCCTTA_dc3hLPS_A8,control,NaN
4,4,AAACATACTCTCCG_dc3hLPS_A8,m_Runx1_2,Runx1
...,...,...,...,...
32772,32772,TTTCTACTGACGAG_dc3hLPS_D9,m_Stat2_4,Stat2
32773,32773,TTTCTACTTCTTAC_dc3hLPS_D9,control,NaN
32774,32774,TTTGACTGAAAAGC_dc3hLPS_D9,m_Rela_1,Rela
32775,32775,TTTGCATGCGACAT_dc3hLPS_D9,control,NaN


In [127]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import scanpy as sc

def compute_grn_from_covariance(adata, threshold=0.05):
    """
    Computes a Gene Regulatory Network (GRN) using gene expression covariance.

    Parameters:
    - adata: AnnData object containing gene expression data.
    - threshold: Minimum absolute covariance value for an edge.

    Returns:
    - edge_index: PyTorch tensor representing the GRN edges.
    """
    expression_matrix = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
    covariance_matrix = np.cov(expression_matrix, rowvar=False)

    edge_list = []
    num_genes = covariance_matrix.shape[0]

    for i in range(num_genes):
        for j in range(i + 1, num_genes):
            if abs(covariance_matrix[i, j]) > threshold:
                edge_list.append((i, j))
                edge_list.append((j, i))  # Undirected graph

    edge_index = torch.tensor(edge_list, dtype=torch.long).T
    return edge_index

# Compute GRN edges
grn_edge_index = compute_grn_from_covariance(preturbed_adata)


import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GeneEmbeddingGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GeneEmbeddingGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize random input features (gene embeddings)
num_genes = preturbed_adata.shape[1]
embedding_dim = 64  # Gene embedding dimension
gene_features = torch.randn(num_genes, embedding_dim)

# Create PyTorch Geometric Data object
gene_graph = Data(x=gene_features, edge_index=grn_edge_index)

# Instantiate and run GNN
gene_gnn = GeneEmbeddingGNN(input_dim=embedding_dim, hidden_dim=128, output_dim=embedding_dim)
perturbation_embeddings = gene_gnn(gene_graph.x, gene_graph.edge_index)

# Convert to DataFrame
perturbation_embeddings_df = pd.DataFrame(perturbation_embeddings.detach().numpy(), index=preturbed_adata.var_names)

print(f" Generated gene embeddings with shape {perturbation_embeddings_df.shape}.")



 Generated gene embeddings with shape (24, 64).


In [128]:
perturbation_embeddings

tensor([[-0.5619,  0.5695, -0.4530,  ..., -0.1940,  0.1096, -0.2489],
        [ 0.3643, -0.4883,  0.4908,  ...,  0.3322,  0.1626,  0.0056],
        [ 0.4872,  0.1893,  0.2931,  ..., -0.2984, -0.1759,  0.7876],
        ...,
        [ 0.4763,  0.7349, -0.9968,  ..., -0.0011, -0.3146, -0.1546],
        [ 0.5294, -0.2087, -0.1362,  ...,  0.5635, -0.5476,  0.0874],
        [ 1.0526,  1.1167, -1.0149,  ...,  0.1186, -0.4439, -0.2328]],
       grad_fn=<AddBackward0>)

In [129]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import scanpy as sc

def compute_grn_from_covariance(adata, threshold=0.05):
    """
    Computes a Gene Regulatory Network (GRN) using gene expression covariance.

    Parameters:
    - adata: AnnData object containing gene expression data.
    - threshold: Minimum absolute covariance value for an edge.

    Returns:
    - edge_index: PyTorch tensor representing the GRN edges.
    """
    expression_matrix = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
    covariance_matrix = np.cov(expression_matrix, rowvar=False)

    edge_list = []
    num_genes = covariance_matrix.shape[0]

    for i in range(num_genes):
        for j in range(i + 1, num_genes):
            if abs(covariance_matrix[i, j]) > threshold:
                edge_list.append((i, j))
                edge_list.append((j, i))  # Undirected graph

    edge_index = torch.tensor(edge_list, dtype=torch.long).T
    return edge_index

# Compute GRN edges
grn_edge_index = compute_grn_from_covariance(adata)


import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GeneEmbeddingGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GeneEmbeddingGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize random input features (gene embeddings)
num_genes = adata.shape[1]
embedding_dim = 64  # Gene embedding dimension
gene_features = torch.randn(num_genes, embedding_dim)

# Create PyTorch Geometric Data object
gene_graph = Data(x=gene_features, edge_index=grn_edge_index)

# Instantiate and run GNN
gene_gnn = GeneEmbeddingGNN(input_dim=embedding_dim, hidden_dim=128, output_dim=embedding_dim)
gene_embeddings = gene_gnn(gene_graph.x, gene_graph.edge_index)

# Convert to DataFrame
gene_embeddings_df = pd.DataFrame(gene_embeddings.detach().numpy(), index=adata.var_names)

print(f"✅ Generated gene embeddings with shape {gene_embeddings_df.shape}.")




✅ Generated gene embeddings with shape (5018, 64).


In [130]:
gene_embeddings

tensor([[ 0.4901,  0.3833,  0.3979,  ..., -0.9739, -0.8685, -0.1473],
        [ 0.0157,  0.0458,  0.8190,  ..., -0.5170,  1.1289, -0.6276],
        [ 1.2202,  0.8001, -0.7026,  ..., -0.2977, -2.5305, -0.3518],
        ...,
        [ 0.0939,  1.3783,  0.2176,  ..., -0.8683, -0.6194, -0.9995],
        [ 0.3055,  0.7999,  0.2410,  ..., -0.0323, -0.7287, -0.2633],
        [-0.3389,  0.8239, -0.6588,  ..., -0.1849,  0.8136, -0.3119]],
       grad_fn=<AddBackward0>)

In [140]:
# Initialize an empty list to store positions of perturbation genes in gene_index
perturbation_positions = []

# Iterate through perturbation indices and find matching positions in gene_index
for perturbation_id in perturbation_index:
    for pos, gene_id in enumerate(gene_index):
        if gene_id == perturbation_id:  # Match found
            perturbation_positions.append(pos)  # Store the position
            break  # Stop after finding the first match to maintain order

print(f"✅ Extracted {len(perturbation_positions)} positions of perturbation genes in gene_index.")
print(f"Perturbation Positions: {perturbation_positions}")

extracted_gene_embeddings = gene_embeddings[perturbation_positions,:]
extracted_gene_embeddings

✅ Extracted 24 positions of perturbation genes in gene_index.
Perturbation Positions: [5004, 5003, 5009, 5015, 5011, 3155, 2113, 707, 5000, 5007, 5016, 4498, 5012, 5005, 5006, 1800, 5013, 5010, 5014, 2025, 5017, 5001, 5002, 5008]


tensor([[-5.5604e-04,  7.5179e-02, -4.9767e-03,  ..., -4.5681e-02,
         -3.9258e-04, -3.9346e-02],
        [-2.2259e-01,  1.7317e-01, -5.5421e-02,  ..., -7.8946e-01,
         -1.3193e+00, -2.9324e-01],
        [-1.7038e-02,  7.3041e-02,  2.4959e-02,  ..., -3.0534e-02,
         -1.2774e-02, -2.2900e-02],
        ...,
        [ 3.5478e-03,  8.4826e-02,  4.4246e-02,  ..., -7.9988e-02,
          8.9703e-03, -7.2632e-02],
        [ 1.5704e-02,  4.7382e-02, -1.8966e-02,  ..., -5.6767e-02,
         -6.3808e-02, -4.8464e-02],
        [-3.3164e-01,  1.1002e+00, -8.7090e-02,  ..., -3.5304e-01,
         -7.8286e-01, -2.2205e-01]], grad_fn=<IndexBackward0>)

In [163]:
import torch
import torch.nn as nn

class CompositionalModule(nn.Module):
    """
    Combines perturbation embeddings with gene embeddings to model interactions.
    """
    def __init__(self, embedding_dim):
        super(CompositionalModule, self).__init__()
        self.fc = nn.Linear(embedding_dim * 2, embedding_dim)  # Compress combined embeddings

    def forward(self, perturbation_embedding, extracted_gene_embedding):
        """
        Forward pass for compositional module:
        - Combines perturbation and gene embeddings.
        - Applies transformation to capture relationships.

        Inputs:
        - perturbation_embedding: Tensor of shape (batch_size, embedding_dim)
        - extracted_gene_embedding: Tensor of shape (batch_size, embedding_dim)

        Output:
        - Tensor of shape (batch_size, embedding_dim)
        """
        combined = torch.cat([perturbation_embedding, extracted_gene_embedding], dim=1)  # Concatenate embeddings
        print(f"🔹 Perturbation embedding shape: {perturbation_embedding.shape}")  # Should be (batch, 64)
        print(f"🔹 Extracted gene embedding shape: {extracted_gene_embedding.shape}")  # Should be (batch, 64)
        
        return self.fc(combined)  # Apply a linear transformation


class CrossGeneDecoder(nn.Module):
    """
    Predicts post-perturbation gene expression based on embeddings.
    """
    def __init__(self, embedding_dim, num_genes):
        super(CrossGeneDecoder, self).__init__()
        self.fc = nn.Linear(embedding_dim, num_genes)  # Maps embedding to gene expression space

    def forward(self, combined_embedding):
        """
        Forward pass for cross-gene decoder:
        - Maps combined embedding to gene expression space.

        Input:
        - combined_embedding: Tensor of shape (batch_size, embedding_dim)

        Output:
        - Tensor of shape (batch_size, num_genes) (Predicted expression)
        """
        return self.fc(combined_embedding)  # Outputs predicted expression


class AutofocusLoss(nn.Module):
    """
    Implements the Autofocus direction-aware loss.
    """
    def __init__(self, gamma=2, lambda_reg=0.1):
        super(AutofocusLoss, self).__init__()
        self.gamma = gamma  # Weight for autofocus loss
        self.lambda_reg = lambda_reg  # Regularization term
        self.mse = nn.MSELoss()  # Standard loss for expression prediction

    def forward(self, predicted_expression, true_expression, perturbation_embedding):
        """
        Computes loss:
        - MSE loss between predicted & true expression
        - Autofocus loss penalizing large perturbations
        - Regularization term for stable training

        Inputs:
        - predicted_expression: Tensor of shape (batch_size, num_genes)
        - true_expression: Tensor of shape (batch_size, num_genes)
        - perturbation_embedding: Tensor of shape (batch_size, embedding_dim)

        Output:
        - Scalar loss value
        """
        mse_loss = self.mse(predicted_expression, true_expression)  # Ensure gradients flow
        focus_loss = torch.mean(torch.abs(perturbation_embedding))  # Encourage stable perturbation embeddings
        
        # Total loss combines prediction error and embedding stability
        total_loss = mse_loss + self.gamma * focus_loss + self.lambda_reg * torch.norm(perturbation_embedding, p=2)
        
        return total_loss  # No .detach(), maintain computational graph



import torch
import numpy as np

# def autofocus_loss(predicted_expression, true_expression, perturbation_names, gamma=2, lambda_reg=0.1, direction_lambda=1e-3):
#     """
#     Adapted Autofocus Loss Function

#     Args:
#         predicted_expression (torch.Tensor): Model output for post-perturbation gene expression.
#         true_expression (torch.Tensor): True post-perturbation gene expression.
#         perturbation_names (list): List of perturbation conditions.
#         gamma (float): Exponent for the autofocus loss.
#         lambda_reg (float): Regularization term for uncertainty handling.
#         direction_lambda (float): Weight for directional loss.

#     Returns:
#         torch.Tensor: Computed loss value.
#     """
#     perturbation_names = np.array(list(set(perturbation_names)))
#     losses = torch.tensor(0.0, requires_grad=True).to(predicted_expression.device)
#     print(len(perturbation_names), predicted_expression.size())
#     for pert in set(perturbation_names):
#         if pert != "control":
#             pert_idx = np.where(perturbation_names == pert)[0]
#             pred_pert = predicted_expression[pert_idx]
#             true_pert = true_expression[pert_idx]

#             # Compute standard MSE loss
#             mse_loss = torch.mean((pred_pert - true_pert) ** (2 + gamma))

#             # Autofocus regularization
#             focus_loss = torch.mean(torch.abs(pred_pert))

#             # Direction-aware regularization
#             dir_loss = direction_lambda * torch.mean((torch.sign(true_pert) - torch.sign(pred_pert)) ** 2)

#             # Combine all components
#             loss = mse_loss + gamma * focus_loss + lambda_reg * dir_loss
#             losses += loss

#     return losses / len(set(perturbation_names))



In [161]:
import pandas as pd
import torch

import torch
import pandas as pd

# Step 1: Add a "symbols" column to adata.obs for matching
adata.obs["symbols"] = adata.obs["perturbation"].str.split("_").str[1]  # Extract gene names from perturbations

# Initialize a list to store mean expression per perturbation
perturbation_mean_expressions = []

# Convert adata.X to dense array (if it's sparse) for easier computation
expression_matrix = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X

# Step 2: Compute mean expression for each perturbation symbol
for symbol in perturbation_symbols:
    # Find indices of cells where perturbation matches this symbol
    matching_cells_mask = adata.obs["symbols"] == symbol

    # Extract expression data for these cells
    matching_expression = expression_matrix[matching_cells_mask]

    # Compute mean expression across all matching cells (1 × 5024 vector)
    mean_expression = matching_expression.mean(axis=0)

    # Store the mean expression
    perturbation_mean_expressions.append(mean_expression)

# Convert to tensor for training
true_expression = torch.tensor(perturbation_mean_expressions, dtype=torch.float32)

# Display final shape
print(f"✅ Computed `true_expression_tensor` with shape: {true_expression.shape}")  # Expected: (24, 5024) or similar


✅ Computed `true_expression_tensor` with shape: torch.Size([24, 5018])


In [167]:
# Hyperparameters
embedding_dim = 64
hidden_dim = 128
gamma = 2
lambda_reg = 0.1
learning_rate = 1e-3
num_epochs = 200

# Initialize Model Components
compositional_module = CompositionalModule(embedding_dim)
cross_gene_decoder = CrossGeneDecoder(embedding_dim, num_genes=adata.shape[1])
loss_fn = AutofocusLoss(gamma, lambda_reg)
optimizer = torch.optim.Adam(
    list(compositional_module.parameters()) + list(cross_gene_decoder.parameters()), lr=learning_rate
)

# Ensure perturbation embeddings require gradients
perturbation_embeddings.requires_grad_(True)

# Training Loop
for epoch in range(num_epochs):
    # optimizer.zero_grad()  # Reset gradients before backprop

    # Sample perturbation and gene embeddings
    perturb_embed = perturbation_embeddings
    gene_embed = extracted_gene_embeddings

    # Combine embeddings
    combined_embedding = compositional_module(perturb_embed, gene_embed)

    # Predict post-perturbation expression
    predicted_expression = cross_gene_decoder(combined_embedding)
    # print(predicted_expression, true_expression, perturb_embed)
    # Compute loss
    loss = loss_fn(predicted_expression, true_expression, perturb_embed)

    # Debug prints
    print(f"Predicted Expression requires grad: {predicted_expression.requires_grad}")
    print(f"Perturbation Embedding requires grad: {perturb_embed.requires_grad}")

    print(predicted_expression.requires_grad)  # Must be True
    print(combined_embedding.requires_grad)  # Must be True
    print(perturb_embed.requires_grad)  # Must be True
    print(gene_embed.requires_grad)  # Must be True
    
    print("true_expression shape:", perturb_embed.shape)
    print("true_expression requires_grad:", true_expression.requires_grad)  # MUST be False
    print("true_expression values (first 5 rows):\n", true_expression[:5])

     # Should be True
    loss.backward()

    optimizer.step()  # Update model parameters

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


🔹 Perturbation embedding shape: torch.Size([24, 64])
🔹 Extracted gene embedding shape: torch.Size([24, 64])
Predicted Expression requires grad: True
Perturbation Embedding requires grad: True
True
True
True
True
true_expression shape: torch.Size([24, 64])
true_expression requires_grad: False
true_expression values (first 5 rows):
 tensor([[0.3439, 0.0469, 0.5488,  ..., 0.2109, 0.2470, 0.7128],
        [0.3190, 0.0383, 0.5597,  ..., 0.2111, 0.2634, 0.6444],
        [0.3317, 0.0233, 0.5922,  ..., 0.2264, 0.2855, 0.7318],
        [0.3090, 0.0508, 0.5396,  ..., 0.2265, 0.2490, 0.7076],
        [0.3064, 0.0327, 0.5491,  ..., 0.1894, 0.2616, 0.6917]])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
import torch
import torch.nn as nn

# Small batch size (3 examples instead of 24 for testing)
batch_size = 3
embedding_dim = 3
num_genes = 3  # Total genes

# Simulated embeddings
perturb_embed = torch.randn(batch_size, embedding_dim, requires_grad=True)  # ✅ Needs grad
extracted_gene_embed = torch.randn(batch_size, embedding_dim, requires_grad=True)  # ✅ Needs grad

# Simulated expression matrices
true_expression = torch.randn(batch_size, num_genes)  # ❌ No need for gradient
predicted_expression = torch.randn(batch_size, num_genes, requires_grad=True)  # ✅ Needs grad
print(perturb_embed, gene_embed)
# Print shapes
print(f"Perturb Embed Shape: {perturb_embed.shape}")
print(f"Gene Embed Shape: {extracted_gene_embed.shape}")
print(f"True Expression Shape: {true_expression.shape}")
print(f"Predicted Expression Shape: {predicted_expression.shape}")


tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True) tensor([[-0.2432, -0.7535,  0.2278],
        [-0.7002,  0.3161,  1.3174],
        [ 1.0295, -1.2459,  0.2643]], requires_grad=True)
Perturb Embed Shape: torch.Size([3, 3])
Gene Embed Shape: torch.Size([3, 3])
True Expression Shape: torch.Size([3, 3])
Predicted Expression Shape: torch.Size([3, 3])


In [ ]:
class CompositionalModule(nn.Module):
    def __init__(self, embedding_dim):
        super(CompositionalModule, self).__init__()
        self.fc = nn.Linear(embedding_dim * 2, embedding_dim)  

    def forward(self, perturbation_embedding, extracted_gene_embedding):
        combined = torch.cat([perturbation_embedding, extracted_gene_embedding], dim=1)  
        output = self.fc(combined)  
        return output  

class CrossGeneDecoder(nn.Module):
    def __init__(self, embedding_dim, num_genes):
        super(CrossGeneDecoder, self).__init__()
        self.fc = nn.Linear(embedding_dim, num_genes)  

    def forward(self, combined_embedding):
        return self.fc(combined_embedding)  

# Initialize modules
compositional_module = CompositionalModule(embedding_dim)
cross_gene_decoder = CrossGeneDecoder(embedding_dim, num_genes)
print(perturb_embed, gene_embed)
# Forward pass
combined_embedding = compositional_module(perturb_embed, extracted_gene_embed)
predicted_expression = cross_gene_decoder(combined_embedding)
print(perturb_embed, gene_embed)
print(f"Combined Embedding Shape: {combined_embedding.shape}")  # Should be (3, 64)
print(f"Predicted Expression Shape: {predicted_expression.shape}")  # Should be (3, 5024)


tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True) tensor([[-0.2432, -0.7535,  0.2278],
        [-0.7002,  0.3161,  1.3174],
        [ 1.0295, -1.2459,  0.2643]], requires_grad=True)
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True) tensor([[-0.2432, -0.7535,  0.2278],
        [-0.7002,  0.3161,  1.3174],
        [ 1.0295, -1.2459,  0.2643]], requires_grad=True)
Combined Embedding Shape: torch.Size([3, 3])
Predicted Expression Shape: torch.Size([3, 3])


In [ ]:
class AutofocusLoss(nn.Module):
    def __init__(self, gamma=2, lambda_reg=0.1):
        super(AutofocusLoss, self).__init__()
        self.gamma = gamma  
        self.lambda_reg = lambda_reg  
        self.mse = nn.MSELoss()  

    def forward(self, predicted_expression, true_expression, perturb_embed):
        mse_loss = self.mse(predicted_expression, true_expression)  # ✅ Shape (3, 5024)
        reg_loss = self.lambda_reg * torch.norm(perturb_embed, p=2)  # ✅ Regularization on (3, 64)
        total_loss = mse_loss + reg_loss
        return total_loss  

import torch
import torch.nn as nn

# Ensure model is in training mode
compositional_module.train()
cross_gene_decoder.train()

# Training Loop
num_epochs = 200
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Reset gradients before each iteration

    # Forward pass
    perturb_embed = perturb_embed  # From Perturbation GNN
    gene_embed = extracted_gene_embed  # From Gene GNN
    # print(perturb_embed, gene_embed)
    combined_embedding = compositional_module(perturb_embed, gene_embed)
    predicted_expression = cross_gene_decoder(combined_embedding)

    # Compute loss
    loss = loss_fn(predicted_expression, true_expression, perturb_embed)

    # Debugging Checks
    if epoch == 0:  # Print checks only for the first iteration
        print(f"✅ Loss at start: {loss.item():.4f}")
        print(f"Loss requires grad? {loss.requires_grad}")
        print(f"Predicted Expression requires grad? {predicted_expression.requires_grad}")
        print(f"Perturbation Embedding requires grad? {perturb_embed.requires_grad}")

    # Backpropagation
    loss.backward(retain_graph=False)  # Only use retain_graph=True if needed

    optimizer.step()  # Update model parameters

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")
        print(perturb_embed)
# Ensure models are in evaluation mode after training
compositional_module.eval()
cross_gene_decoder.eval()
print("✅ Training complete!")


✅ Loss at start: 1.0126
Loss requires grad? True
Predicted Expression requires grad? True
Perturbation Embedding requires grad? True
Epoch 0: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True)
Epoch 10: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True)
Epoch 20: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True)
Epoch 30: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True)
Epoch 40: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031,  1.6289],
        [ 0.6273,  0.2213, -1.7466]], requires_grad=True)
Epoch 50: Loss = 1.0126
tensor([[ 0.5125, -1.0001, -0.0074],
        [-1.4243,  0.0031, 

In [146]:
from gears import PertData, GEARS

# get data
pert_data = PertData(r"C:\Users\W\Desktop\Assignments\Data Science\project\repreduce 1\data_visual\adata_1")
# load dataset in paper: norman, adamson, dixit.
pert_data.load(data_name = 'norman')
# specify data split
pert_data.prepare_split(split = 'simulation', seed = 1)
# get dataloader with batch size
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

# set up and train a model
gears_model = GEARS(pert_data, device = 'cuda:8')
gears_model.model_initialize(hidden_size = 64)
gears_model.train(epochs = 20)

# save/load model
gears_model.save_model('gears')
gears_model.load_pretrained('gears')

# predict
gears_model.predict([['CBL', 'CNN1'], ['FEV']])
gears_model.GI_predict(['CBL', 'CNN1'], GI_genes_file=None)

Downloading...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\W\\Desktop\\Assignments\\Data Science\\project\\repreduce 1\\data_visual\\adata_1\\gene2go_all.pkl'